# Autograde a students' solution

**These instructions are currently incomplete**.

Let's say that students have turned in their notebooks with a special naming convention of `Lastname/Problem N.ipynb`, where `N` is the problem number and `Lastname` is a directory named with the student's last name. In this example, we have a few submitted assignments:

* [Bitdiddle/Problem 1.ipynb](grade_example/submitted/Bitdiddle/Problem 1.ipynb)
* [Bitdiddle/Problem 2.ipynb](grade_example/submitted/Bitdiddle/Problem 2.ipynb)
* [Hacker/Problem 1.ipynb](grade_example/submitted/Hacker/Problem 1.ipynb)
* [Hacker/Problem 2.ipynb](grade_example/submitted/Hacker/Problem 2.ipynb)

In [1]:
cd grade_example

/Users/jhamrick/project/tools/nbgrader/docs/user_guide/grade_example


In [2]:
ls -R

nbgrader_autograde_config.py  submitted/
nbgrader_formgrade_config.py

./submitted:
Bitdiddle/ Hacker/

./submitted/Bitdiddle:
Problem 1.ipynb  Problem 2.ipynb

./submitted/Hacker:
Problem 1.ipynb  Problem 2.ipynb


## Initialize the database

Before we can actually start grading, we need to actually record who the students are. We can do this using the API provided by nbgrader, which provides access to a MongoDB to store information about students and their grades:

In [3]:
# create a connection to the MongoDB using the nbgrader API
from nbgrader.api import Gradebook, Student, Assignment
gb = Gradebook("example")

In [4]:
# drop whatever is in the database currently
gb.client.drop_database("example")

In [5]:
# create some students and add them to the database
s1 = Student(student_id="Bitdiddle", first_name="Ben", last_name="Bitdiddle")
s2 = Student(student_id="Hacker", first_name="Alyssa", last_name="Hacker")
s3 = Student(student_id="Reasoner", first_name="Louis", last_name="Reasoner")
gb.add_student(s1)
gb.add_student(s2)
gb.add_student(s3)
gb.students

[{"first_name": "Ben", "_id": "44885f48-7d5c-4571-a949-359ab1c49f2a", "last_name": "Bitdiddle", "email": null, "student_id": "Bitdiddle"},
 {"first_name": "Alyssa", "_id": "b62d36b5-dd7e-43c0-9584-e36c3e9ad643", "last_name": "Hacker", "email": null, "student_id": "Hacker"},
 {"first_name": "Louis", "_id": "f32757c9-62b2-43d1-b0f5-48594131112d", "last_name": "Reasoner", "email": null, "student_id": "Reasoner"}]

We also must create a record for the assignment, which will be associated with the autograded notebooks:

In [6]:
# create the assignment and add it to the database
a = Assignment(assignment_id="Problem Set 1", duedate="January 1, 2014")
gb.add_assignment(a)
gb.assignments

[{"_id": "7f6151eb-7a90-4c74-ad33-d9b37f0fd64c", "duedate": "January 1, 2014", "assignment_id": "Problem Set 1"}]

## Run the autograder

Now that we've set up the database, we can grade the assignments.

First, we specify the configuration file for `nbgrader autograde`:

In [7]:
!cat nbgrader_autograde_config.py

c = get_config()

import os
c.AutogradeApp.notebooks = ['submitted']
c.AutogradeApp.output_dir = os.path.join(os.getcwd(), 'autograded')
c.AutogradeApp.recursive = True
c.FindStudentID.regexp = r"submitted/(?P<student_id>.+)/.*.ipynb"
c.SaveAutoGrades.assignment_id = "Problem Set 1"
c.SaveAutoGrades.db_name = "example"


This config file says that we should convert the files in the `submitted` directory, and save the autograded files to a new directory called `autograded`. We will do this *recursively*, so that the autograder will look into the directories with the students' names. We additionally specify a regular expression which tells the grader how to determine the student ID. In this case, the ID is just their last name. Finally, we tell the autograder what assignment these notebooks are a part of, and what database the grades should be saved into.

Now, to grade the assignments, all we have to do is:

In [8]:
!nbgrader autograde

[AutogradeApp] Using existing profile dir: '/tmp/nbgrader/profile_default'
[AutogradeApp] Directory tree prefix: submitted/
[AutogradeApp] Changing to directory: submitted/Bitdiddle
[AutogradeApp] Converting notebook Problem 1.ipynb to notebook
[AutogradeApp] Support files will be in Problem 1_files/


[AutogradeApp] Student ID: Bitdiddle


[AutogradeApp] Executing notebook with kernel: python3


[AutogradeApp] Writing output to directory: autograded/Bitdiddle
[AutogradeApp] Writing 51966 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/grade_example/autograded/Bitdiddle/Problem 1.ipynb
[AutogradeApp] Changing to directory: submitted/Bitdiddle
[AutogradeApp] Converting notebook Problem 2.ipynb to notebook
[AutogradeApp] Support files will be in Problem 2_files/


[AutogradeApp] Student ID: Bitdiddle
[AutogradeApp] Executing notebook with kernel: python3


[AutogradeApp] Writing output to directory: autograded/Bitdiddle
[AutogradeApp] Writing 2721 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/grade_example/autograded/Bitdiddle/Problem 2.ipynb
[AutogradeApp] Changing to directory: submitted/Hacker
[AutogradeApp] Converting notebook Problem 1.ipynb to notebook
[AutogradeApp] Support files will be in Problem 1_files/


[AutogradeApp] Student ID: Hacker
[AutogradeApp] Executing notebook with kernel: python3


[AutogradeApp] Writing output to directory: autograded/Hacker
[AutogradeApp] Writing 24113 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/grade_example/autograded/Hacker/Problem 1.ipynb
[AutogradeApp] Changing to directory: submitted/Hacker
[AutogradeApp] Converting notebook Problem 2.ipynb to notebook
[AutogradeApp] Support files will be in Problem 2_files/
[AutogradeApp] Student ID: Hacker
[AutogradeApp] Executing notebook with kernel: python3


[AutogradeApp] Writing output to directory: autograded/Hacker
[AutogradeApp] Writing 2813 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/grade_example/autograded/Hacker/Problem 2.ipynb


Now if we look at the `autograded` directory, we see:

* [Bitdiddle/Problem 1.ipynb](grade_example/autograded/Bitdiddle/Problem 1.ipynb)
* [Bitdiddle/Problem 2.ipynb](grade_example/autograded/Bitdiddle/Problem 2.ipynb)
* [Hacker/Problem 1.ipynb](grade_example/autograded/Hacker/Problem 1.ipynb)
* [Hacker/Problem 2.ipynb](grade_example/autograded/Hacker/Problem 2.ipynb)

In [9]:
ls -R autograded

Bitdiddle/ Hacker/

autograded/Bitdiddle:
Problem 1.ipynb  Problem 2.ipynb

autograded/Hacker:
Problem 1.ipynb  Problem 2.ipynb
